# Vehicle Detection Project
#### Implementation of Vehicle Detection Project for training dataset. The output is generate the parameters.p (It will be used by Vehicle-detection-implementation.ipynb)

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import fnmatch
from PIL import Image
from skimage.feature import hog
%matplotlib inline


import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from lesson_functions import *

import pickle


In [2]:
def recursive_glob(rootdir='.', pattern='*'):
    """Search recursively for files matching a specified pattern.
    
    Adapted from http://stackoverflow.com/questions/2186525/use-a-glob-to-find-files-recursively-in-python
    """

    matches = []
    for root, dirnames, filenames in os.walk(rootdir):
        for filename in fnmatch.filter(filenames, pattern):
            matches.append(os.path.join(root, filename))

    return matches

In [3]:
# Read in cars and notcars
images = recursive_glob(pattern='*.png')
cars = []
notcars = []
for image in images:
    if 'image' in image:
        cars.append(image)
    elif 'extra' in image:
        notcars.append(image)
        
print('cars:',len(cars),'notcars:',len(notcars))

cars: 8789 notcars: 8968


In [4]:

########################################
####################################
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
orient = 9
pix_per_cell = 8
cell_per_block = 2
spatial_size = (32,32)
hist_bins = 16
hog_channel = 'ALL'


car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler

X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Train:', len(y_train), 'test:',len(y_test))
print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()

t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()



Train: 14205 test: 3552
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8412
52.22 Seconds to train SVC...
Test Accuracy of SVC =  0.9893


#### the output is the parameters dictionary

In [5]:
parameters={}
parameters['spatial_feat'] = spatial_feat # Spatial features on or off
parameters['hist_feat'] = hist_feat # Histogram features on or off
parameters['hog_feat'] = hog_feat # HOG features on or off
parameters['orient'] = orient
parameters['pix_per_cell'] = pix_per_cell
parameters['cell_per_block'] = cell_per_block
parameters['spatial_size'] = spatial_size
parameters['hist_bins'] = hist_bins
parameters['hog_channel'] = hog_channel
parameters['X_scaler'] = X_scaler
parameters['svc']      = svc
parameters['color_space']=color_space

with open('parameters.p', 'wb') as handle:
    pickle.dump(parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)